In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/Users/jacob.perius/psa_segment_testing/Incrementality Halfway Analysis/data/data_1_7_24/SubmissionData_20250106_000000000002.csv')

print(df.columns)
df

In [ ]:
df = df[df['is_current_psa_dealer'] == False]
df

In [ ]:
df = df[['customer_id', 'submission_id', 'online_completion_date', 'submission_amount_quoted_initial', 'shipping_zipcode', 'submitter_category', 'total_quantity']].copy()
df['online_completion_date'] = pd.to_datetime(df['online_completion_date'])
df['shipping_zipcode'] = df['shipping_zipcode'].str.split('-').str[0].str.zfill(5)

df = df[df['online_completion_date'] >= '2024-11-03'].copy()

df['time_post'] = np.where(df['online_completion_date'] <= '2024-12-12', 0, 1)

df.head()

In [ ]:
zip_to_dma_df = pd.read_csv('/Users/jacob.perius/psa_segment_testing/incrementality_testing/data/ENV _ Census _ ZIP to DMA.csv')
zip_to_dma_df['zip_code'] = zip_to_dma_df['zip_code'].astype('str').str.zfill(5)

zip_to_dma_df.head()

In [ ]:
submissions_with_dma = pd.merge(df, zip_to_dma_df, how='left', left_on='shipping_zipcode', right_on='zip_code')
submissions_with_dma = submissions_with_dma.drop('zip_code', axis=1)

submissions_with_dma.head()

In [ ]:
group_df = pd.read_csv('/Users/jacob.perius/psa_segment_testing/incrementality_testing/final_groups/final_groups.csv')

group_df.head()

In [ ]:
submissions_with_group = pd.merge(submissions_with_dma, group_df, how='left', on=['dma_code', 'dma_description'])

submissions_with_group

In [ ]:
date_group_df = submissions_with_group.groupby(['online_completion_date', 'time_post', 'group']).agg({
    'total_quantity': 'sum'
}).reset_index()

date_group_df['online_completion_date'] = pd.to_datetime(date_group_df['online_completion_date'])

date_group_df['dma_split'] = (date_group_df['group'] == 'Group B').astype(int)

date_group_df.head()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=date_group_df[date_group_df['group'] == 'Group A']['online_completion_date'].to_list(),
    y=date_group_df[date_group_df['group'] == 'Group A']['total_quantity'].to_list(),
    mode='lines',
    name='Group A - Control',
    line=dict(width=1, color='blue'),
    showlegend=True
))

fig.add_trace(go.Scatter(
    x=date_group_df[date_group_df['group'] == 'Group B']['online_completion_date'].to_list(),
    y=date_group_df[date_group_df['group'] == 'Group B']['total_quantity'].to_list(),
    mode='lines',
    name='Group B - Treatment',
    line=dict(width=1, color='red'),
    showlegend=True
))

fig.add_trace(go.Scatter(
    x=['2024-12-12', '2024-12-12'],
    y=[min(date_group_df[date_group_df['group'] == 'Group A']['total_quantity']), max(date_group_df[date_group_df['group'] == 'Group B']['total_quantity'].to_list())],
    mode='lines',
    name='Activation',
    line=dict(color='green', dash='dash')
))

fig.update_layout(
    title='PSA Incrementality Test Halfway Analysis (11/3/24 - 1/6/25)',
    xaxis_title='Date',
    yaxis_title='Total Quantity - Submitted Items',
    template='plotly_white',
    legend=dict(orientation='v', x=1, xanchor='center'),
    width=1300,
    height=600
)

fig.show()
#fig.write_html('/Users/jacob.perius/psa_segment_testing/Incrementality Halfway Analysis/PSA Incrementality Halfway Analysis.html')

In [ ]:
seasonality_df = date_group_df.groupby('online_completion_date')['total_quantity'].sum().reset_index(drop=False).copy()
seasonality_df = seasonality_df[['online_completion_date', 'total_quantity']].copy()
seasonality_df['online_completion_date'] = pd.to_datetime(seasonality_df['online_completion_date'])

seasonality_df.head()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition = seasonal_decompose(seasonality_df['total_quantity'], period=7, model='additive')

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

seasonality_df.loc[:, 'seasonality'] = seasonal

date_group_df['dow'] = date_group_df['online_completion_date'].dt.strftime('%A')

seasonality_df = pd.merge(
    seasonality_df,
    date_group_df[['online_completion_date', 'dow']],
    on='online_completion_date',
    how='left'  # Use 'left' to preserve all rows in seasonality_df
)

dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
seasonality_df['dow'] = pd.Categorical(
    seasonality_df['dow'],
    categories=dow_order,
    ordered=True
)

print(seasonality_df.shape)
seasonality_df.head(3)

In [ ]:
date_group_df2 = pd.merge(submissions_with_group, seasonality_df.drop('total_quantity', axis=1), how='left', on='online_completion_date')

date_group_df2.head()

In [ ]:
date_group_df2['dma_split'] = (date_group_df2['group'] == 'Group B').astype(int)

date_group_df2.head()

In [ ]:
regression_df = date_group_df2.groupby(['online_completion_date', 'time_post', 'dma_split', 'group', 'submitter_category']).agg({
    'seasonality': 'first',
    'total_quantity': 'sum'
}).reset_index()

regression_df

In [ ]:
before_df = regression_df[regression_df['time_post'] == 0].copy()
after_df = regression_df[regression_df['time_post'] == 1].copy()

before_sums_df = before_df.groupby('group')['total_quantity'].sum().reset_index()
after_sums_df = after_df.groupby('group')['total_quantity'].sum().reset_index()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=before_sums_df[before_sums_df['group'] == 'Group A']['group'].to_list(),
    y=before_sums_df[before_sums_df['group'] == 'Group A']['total_quantity'].to_list(),
    name='Group A - Control',
    marker=dict(color='red', line=dict(width=1)),
    showlegend=True
))

fig.add_trace(go.Bar(
    x=before_sums_df[before_sums_df['group'] == 'Group B']['group'].to_list(),
    y=before_sums_df[before_sums_df['group'] == 'Group B']['total_quantity'].to_list(),
    name='Group B - Treated Group',
    marker=dict(color='blue', line=dict(width=1)),
    showlegend=True
))

fig.update_layout(
    title='Incrementality Month Before (11/3/24 - 12/3/24) Submissions',
    xaxis_title='Group',
    yaxis_title='Submissions',
    template='plotly_white',
    legend=dict(orientation='v', x=1, xanchor='center'),
    width=900,
    height=600
)

fig.show()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=after_sums_df[after_sums_df['group'] == 'Group A']['group'].to_list(),
    y=after_sums_df[after_sums_df['group'] == 'Group A']['total_quantity'].to_list(),
    name='Group A - Control',
    marker=dict(color='red', line=dict(width=1)),
    showlegend=True
))

fig.add_trace(go.Bar(
    x=after_sums_df[after_sums_df['group'] == 'Group B']['group'].to_list(),
    y=after_sums_df[after_sums_df['group'] == 'Group B']['total_quantity'].to_list(),
    name='Group B - Treated Group',
    marker=dict(color='blue', line=dict(width=1)),
    showlegend=True
))

fig.update_layout(
    title='Incrementality Month Before (12/3/24 - 1/6/25) Submissions',
    xaxis_title='Group',
    yaxis_title='Submissions',
    template='plotly_white',
    legend=dict(orientation='v', x=1, xanchor='center'),
    width=900,
    height=600
)

fig.show()

In [ ]:
platform_spend_data = pd.read_csv('/Users/jacob.perius/psa_segment_testing/Incrementality Halfway Analysis/data/ad_platform_spend.csv')

platform_spend_data['Date'] = pd.to_datetime(platform_spend_data['Date'])

platform_spend_data.head()

In [ ]:
regression_df = pd.merge(regression_df, platform_spend_data, how='left', left_on='online_completion_date', right_on='Date')

regression_df.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
encoded_array = encoder.fit_transform(regression_df[['submitter_category']]).toarray()
submitter_type_one_hot = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(['submitter_category']))

regression_df = pd.concat([regression_df, submitter_type_one_hot], axis=1)

regression_df.columns = regression_df .columns.str.replace(" ", "_")

regression_df

In [ ]:
print(regression_df['seasonality'].describe())
print(regression_df['seasonality'].isna().sum()) 

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
regression_df[['seasonality', 'Bing_Spend', 'Google_Spend', 'Meta_Spend', 'Reddit_Spend', 'Snapchat_Spend', 'StackAdapt_Spend', 'Tiktok_Spend']] = scaler.fit_transform(regression_df[['seasonality', 'Bing_Spend', 'Google_Spend', 'Meta_Spend', 'Reddit_Spend', 'Snapchat_Spend', 'StackAdapt_Spend', 'Tiktok_Spend']])

regression_df

<h2>Basic OLS Regression</h2>

In [ ]:
print(regression_df['total_quantity'].describe())

In [ ]:
print(regression_df['total_quantity'].mean())
print(regression_df['total_quantity'].var())

In [ ]:
from statsmodels.formula.api import ols

regression_df['interaction'] = regression_df['dma_split'] * regression_df['time_post']

model = ols('total_quantity ~ dma_split + time_post + interaction + seasonality + submitter_category_New_Submitter + submitter_category_Reactivated_Submitter + Bing_Spend + Google_Spend + Meta_Spend + Reddit_Spend + Snapchat_Spend + StackAdapt_Spend + Tiktok_Spend', data=regression_df).fit()
print(model.summary())
print(model.aic)

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as stats

stats.probplot(model.resid, dist="norm", plot=plt)
plt.title('OLS Linear Regression Residuals')
plt.show()

In [ ]:
regression_df

<h2>Negative Binomial Modeling</h2>

In [ ]:
import statsmodels.api as sm

y = regression_df['total_quantity']
X = sm.add_constant(regression_df[['dma_split', 'time_post', 'interaction', 'seasonality', 'submitter_category_New_Submitter', 'submitter_category_Reactivated_Submitter', 'Bing_Spend', 'Google_Spend', 'Meta_Spend', 'Reddit_Spend', 'Snapchat_Spend', 'StackAdapt_Spend', 'Tiktok_Spend']])

In [ ]:
alphas = np.linspace(0.01, 2, 10)  # Test a range of alpha values
aic_values = [
    sm.GLM(y, X, family=sm.families.NegativeBinomial(alpha=alpha)).fit().aic
    for alpha in alphas
]
initial_alpha = alphas[np.argmin(aic_values)] 

initial_alpha

In [ ]:
from scipy.optimize import minimize

def aic_objective_function(alpha, y, X):
    alpha = max(0.01, abs(alpha))
    try:
        model = sm.GLM(y, X, family=sm.families.NegativeBinomial(alpha=alpha)).fit()
        print(model.aic)
        return model.aic
    except:
        return np.inf

initial_alpha = 0.23
result = minimize(aic_objective_function, initial_alpha, args=(y, X), method='L-BFGS-B', bounds=[(0.01, None)])

optimal_alpha = result.x[0]

print(optimal_alpha)

In [ ]:
model2 = sm.GLM(y, X, family=sm.families.NegativeBinomial(alpha=optimal_alpha))
result = model2.fit(cov_type="HC1")

print(result.summary())
print(result.aic)

<h2>Statistical Power Testing</h2>

In [ ]:
group_counts = regression_df.groupby(['time_post', 'dma_split', 'submitter_category']).size().reset_index(name='count')
print(group_counts)

In [ ]:
group_counts['proportion'] = group_counts['count'] / group_counts['count'].sum()
print(group_counts)

In [ ]:
from statsmodels.stats.power import TTestIndPower

treated_post = regression_df[(regression_df['dma_split'] == 1) & (regression_df['time_post'] == 1)]
n_treated_post = len(treated_post)
print(n_treated_post)

analysis = TTestIndPower()
effect_size = result.params['interaction'] / result.bse['interaction']
alpha = 0.05
power = 0.8

# Power for subgroup
subgroup_power = analysis.solve_power(effect_size=effect_size, alpha=alpha, nobs1=n_treated_post, ratio=1)
print(f"Power for treated post-treatment group: {subgroup_power:.2f}")

In [ ]:
effect_size = result.params['interaction'] / np.sqrt(result.bse['interaction'])
alpha = 0.05
power = 0.8
n_obs = len(regression_df)

# Power calculation
analysis = TTestIndPower()
sample_size = analysis.solve_power(effect_size=effect_size, alpha=alpha, power=power, ratio=1)
print(f"Required sample size for 80% power: {sample_size:.0f}")
print(f"Current sample size: {n_obs}")

In [ ]:
treated_post = regression_df[(regression_df['dma_split'] == 1) & (regression_df['time_post'] == 1)]
n_treated_post = len(treated_post)
print(f"Number of treated samples post-treatment: {n_treated_post}")

In [ ]:
treated_proportion = len(regression_df[regression_df['dma_split'] == 1]) / len(regression_df)
post_treatment_proportion = len(regression_df[regression_df['time_post'] == 1]) / len(regression_df)

# Estimated required treated post-treatment size
required_treated_post = sample_size * treated_proportion * post_treatment_proportion
print(f"Required treated post-treatment sample size: {required_treated_post:.0f}")
print(f"Actual treated post-treatment sample size: {n_treated_post}")

<h2>Heteroskedacisity Check</h2>

In [ ]:
plt.scatter(result.fittedvalues, result.resid_response, alpha=0.6)
plt.axhline(0, color="red", linestyle="--")
plt.xlabel("Fitted Values (Predicted Mean)")
plt.ylabel("Residuals (Response)")
plt.title("Residuals vs Fitted Values")
plt.show()

In [ ]:
from statsmodels.stats.diagnostic import het_breuschpagan

exog = result.model.exog  # Predictors (X)
bp_test = het_breuschpagan(result.resid_response, exog)

bp_stat, bp_pval, _, _ = bp_test
print(f"Breusch-Pagan Test Statistic: {bp_stat}")
print(f"p-value: {bp_pval}")

<h1>Bayesian Modeling</h1>

Note 1

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import pymc as pm
import arviz as az

test_group = regression_df['dma_split'].values  # Top-level group (treatment/control)
submitter_groups = regression_df['submitter_category'].astype('category').cat.codes.values

with pm.Model() as model:
    # Priors for global effects
    intercept = pm.Normal("Intercept", mu=0, sigma=10)
    global_treatment_effect_beta = pm.Normal("Global_Treatment_Effect", mu=0, sigma=2)
    test_group_baseline_beta = pm.Normal("Test_Group_Baseline_Effect", mu=0, sigma=2, shape=len(np.unique(test_group)))  # Treatment/control group baseline effect
    time_post_beta = pm.Normal("Time_Post_Effect", mu=0, sigma=2)
    
    # Hierarchical structure for subcategories
    submitter_category_effect_beta = pm.Normal("Submitter_Category_Baseline_Effect", mu=0, sigma=1, shape=len(np.unique(submitter_groups)))
    submitter_category_treatment_effect_beta = pm.Normal("Submitter_Category_Treatment_Effect", mu=0, sigma=1, shape=len(np.unique(submitter_groups)))
    
    # Priors for fixed effects
    seasonality_effect_beta = pm.Normal("Seasonality_Effect", mu=0, sigma=2)
    bing_spend_beta = pm.LogNormal("Bing_Spend_Effect", mu=0, sigma=2)
    google_spend_beta = pm.LogNormal("Google_Spend_Effect", mu=0, sigma=2)

    alpha_meta_spend = pm.HalfNormal("Alpha_Param_Meta_Spend", sigma=2)
    beta_meta_spend = pm.HalfNormal("Beta_Param_Meta_Spend", sigma=1)
    meta_spend_beta = pm.Beta("Meta_Spend_Effect", alpha=alpha_meta_spend, beta=beta_meta_spend)

    reddit_spend_beta = pm.Normal("Reddit_Spend_Effect", mu=0, sigma=2)
    snapchat_spend_beta = pm.LogNormal("Snapchat_Spend_Effect", mu=0, sigma=2)
    stackadapt_spend_beta = pm.LogNormal("StackAdapt_Spend_Effect", mu=0, sigma=2)
    tiktok_spend_beta = pm.LogNormal("TikTok_Spend_Effect", mu=0, sigma=2)

    neg_binom_alpha = pm.Exponential("Alpha_Param - Negative Binomial", 1.0)

    interaction = regression_df['interaction'].to_numpy()

    mu = pm.math.exp(
        intercept
        + test_group_baseline_beta[test_group]
        + global_treatment_effect_beta * interaction
        + submitter_category_effect_beta[submitter_groups]  # Subcategory effect
        + submitter_category_treatment_effect_beta[submitter_groups] * interaction  # Subcategory-specific treatment effect
        + seasonality_effect_beta * regression_df['seasonality'].values # Fixed seasonality effect
        + time_post_beta * regression_df['time_post'].values
        + bing_spend_beta * regression_df['Bing_Spend'].values
        + google_spend_beta * regression_df['Google_Spend'].values
        + meta_spend_beta * regression_df['Meta_Spend'].values
        + reddit_spend_beta * regression_df['Reddit_Spend'].values
        + snapchat_spend_beta * regression_df['Snapchat_Spend'].values
        + stackadapt_spend_beta * regression_df['StackAdapt_Spend'].values
        + tiktok_spend_beta * regression_df['Tiktok_Spend'].values
    )

    # Negative Binomial likelihood
    y_obs = pm.NegativeBinomial("y_obs", mu=mu, alpha=neg_binom_alpha, observed=regression_df['total_quantity'].values)

    trace = pm.sample(2000, tune=1000, target_accept=0.95, return_inferencedata=True, cores=4, chains=4, random_seed=42)
    log_likelihood = pm.compute_log_likelihood(trace)

idata = az.convert_to_inference_data(trace, log_likelihood=log_likelihood)

summary = az.summary(trace, hdi_prob=0.95)

summary = summary.rename(index={'Test_Group_Baseline_Effect[0]': 'Baseline Group A (Control) Effect', 'Test_Group_Baseline_Effect[1]': 'Baseline Group B (Treated) Effect',
                               'Submitter_Category_Baseline_Effect[0]': 'Baseline New Submitter Effect', 'Submitter_Category_Baseline_Effect[1]': 'Baseline Reactivated Submitter Efect',
                               'Submitter_Category_Baseline_Effect[2]': 'Baseline Repeat Submitter Effect', 'Submitter_Category_Treatment_Effect[0]': 'New Submitter Treatment Effect',
                               'Submitter_Category_Treatment_Effect[1]': 'Reactivated Submitter Treatment Effect',
                               'Submitter_Category_Treatment_Effect[2]': 'Repeat Submitter Treatment Effect',
                               'Seasonality_Effect': 'Seasonality Effect'})

summary['interval_width'] = abs(summary['hdi_2.5%'] - summary['hdi_97.5%'])

summary

Note 2

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

print(trace.posterior["Test_Group_Baseline_Effect"].shape)

group_a_effect = trace.posterior["Test_Group_Baseline_Effect"][:, :, 0].values.flatten()
group_b_effect = trace.posterior["Test_Group_Baseline_Effect"][:, :, 1].values.flatten()

interaction_effect = trace.posterior["Global_Treatment_Effect"].values.flatten()

group_a_before = group_a_effect
group_a_after = group_a_effect + interaction_effect

group_b_before = group_b_effect
group_b_after = group_b_effect + interaction_effect

plt.figure(figsize=(12, 6))

sns.kdeplot(group_a_before, label="Group A Before Treatment", fill=True, alpha=0.5)
sns.kdeplot(group_b_before, label="Group B Before Treatment", fill=True, alpha=0.5)

plt.xlabel("Effect Size", fontsize=14)
plt.ylabel("Density", fontsize=14)
plt.title("Posterior Distributions: Group A and B (Before Treatment)", fontsize=16)
plt.legend(loc="best", fontsize=12)

plt.tight_layout()
plt.show()


plt.figure(figsize=(12, 6))

sns.kdeplot(group_a_after, label="Group A After Treatment", fill=True, alpha=0.5)
sns.kdeplot(group_b_after, label="Group B After Treatment", fill=True, alpha=0.5)

plt.xlabel("Effect Size", fontsize=14)
plt.ylabel("Density", fontsize=14)
plt.title("Posterior Distributions: Group A and B (After Treatment)", fontsize=16)
plt.legend(loc="best", fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
effects = trace.posterior["Submitter_Category_Treatment_Effect"]

plt.figure(figsize=(10, 6))
for i, group_name in enumerate(["New Submitter", "Reactivated Submitter", "Repeat Submitter"]):
    sns.kdeplot(
        effects.sel(Submitter_Category_Treatment_Effect_dim_0=i).values.flatten(),
        label=f"{group_name}",
        fill=True,
    )

plt.axvline(0, color="black", linestyle="--", linewidth=1, label="No Effect")
plt.xlabel("Treatment Effect")
plt.ylabel("Density")
plt.title("Posterior Distributions of Treatment Effects")
plt.legend()
plt.show()

In [ ]:
az.plot_posterior(trace, var_names=["Submitter_Category_Treatment_Effect"], hdi_prob=0.95)

In [ ]:
'''
effects_means = trace.posterior["Submitter_Category_Treatment_Effect"].mean(dim=["chain", "draw"]).values
effects_hdi = az.hdi(trace, var_names=["Submitter_Category_Treatment_Effect"], hdi_prob=0.95)

plt.figure(figsize=(8, 6))
for i, group_name in enumerate(["New Submitter", "Reactivated Submitter", "Repeat Submitter"]):
    hdi_lower = effects_hdi["Submitter_Category_Treatment_Effect"].sel(Submitter_Category_Treatment_Effect_dim_0=i)["hdi_2.5%"]
    hdi_upper = effects_hdi["Submitter_Category_Treatment_Effect"].sel(Submitter_Category_Treatment_Effect_dim_0=i)["hdi_97.5%"]
    plt.errorbar(
        effects_means[i],
        i,
        xerr=[[effects_means[i] - hdi_lower], [hdi_upper - effects_means[i]]],
        fmt="o",
        label=f"{group_name}",
    )

plt.axvline(0, color="black", linestyle="--", label="No Effect")
plt.xlabel("Treatment Effect")
plt.yticks(range(len(["New Submitter", "Reactivated Submitter", "Repeat Submitter"])), ["New Submitter", "Reactivated Submitter", "Repeat Submitter"])
plt.title("Treatment Effects with Credible Intervals")
plt.legend()
plt.show()
'''

<h2>Model Diagnostics</h2>

In [ ]:
waic = az.waic(trace)
print(waic)

In [ ]:
# Posterior predictive plots compare model predictions to observed data and can include treatment effects implicitly.

ppc = pm.sample_posterior_predictive(trace, model=model)
idata.extend(pm.sample_posterior_predictive(trace, model=model, return_inferencedata=True))
az.plot_ppc(idata, figsize=(8, 6))

	Consider plotting residuals (observed - predicted) to identify systematic deviations.